In [1]:
!pip install gensim==4.0.1

In [2]:
!pip install lime

     |████████████████████████████████| 275 kB 3.3 MB/s eta 0:00:01
  Created wheel for lime: filename=lime-0.2.0.1-py3-none-any.whl size=283845 sha256=ceb53e0149528a2f61422f411251e477f43c77806ac1b309c35c63beab9aa627
  Stored in directory: /Users/swethapola/Library/Caches/pip/wheels/e6/a6/20/cc1e293fcdb67ede666fed293cb895395e7ecceb4467779546
Successfully built lime


In [3]:
import nltk
#import lime
from nltk.corpus import brown
from nltk.data import find
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.layers import Embedding, Input, Dense, Lambda, Dropout
from tensorflow.keras.models import Model
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report

In [4]:
# %%capture
# !wget https://raw.githubusercontent.com/kashkambath/cvd-reddit-nlp-analysis/swetha/data/NN_Data.csv

In [5]:
df = pd.read_csv('data/NN_Data_2.0_cleaned_2.csv')
if 'Unnamed: 0' in df.columns:
  df = df.drop(columns=['Unnamed: 0'])
df = df[df['text'].notna()] # remove missing rows
df.head()

,text,outcome
0,My dad had a heart attack and spent over 7 min...,1.0
1,I had dangerous open heart surgery due to mult...,1.0
2,"In June of 2020, my Dad collapsed in our yard ...",1.0
3,"after my twin aunts were born, my grandma beca...",1.0
4,Not me but my mother is the most severe case I...,1.0


In [6]:
df["no_punct"] = df['text'].str.replace('[^\w\s]','')
df['str_list'] = df.no_punct.apply(lambda s: s.split(' '))

In [7]:
df.head()

,text,outcome,no_punct,str_list
0,My dad had a heart attack and spent over 7 min...,1.0,My dad had a heart attack and spent over 7 min...,"[My, dad, had, a, heart, attack, and, spent, o..."
1,I had dangerous open heart surgery due to mult...,1.0,I had dangerous open heart surgery due to mult...,"[I, had, dangerous, open, heart, surgery, due,..."
2,"In June of 2020, my Dad collapsed in our yard ...",1.0,In June of 2020 my Dad collapsed in our yard w...,"[In, June, of, 2020, my, Dad, collapsed, in, o..."
3,"after my twin aunts were born, my grandma beca...",1.0,after my twin aunts were born my grandma becam...,"[after, my, twin, aunts, were, born, my, grand..."
4,Not me but my mother is the most severe case I...,1.0,Not me but my mother is the most severe case I...,"[Not, me, but, my, mother, is, the, most, seve..."


In [8]:
df['outcome'].value_counts()

0.0    748
1.0    697
Name: outcome, dtype: int64

In [9]:
men = df[df["outcome"]==0].sample(n=697, random_state=1) #just for the sake of balancing
women = df[df["outcome"]==1]

df = pd.concat([men,women])
df.head()

,text,outcome,no_punct,str_list
1997,Im so so sorry for your loss. I feel your pain...,0.0,Im so so sorry for your loss I feel your pain ...,"[Im, so, so, sorry, for, your, loss, I, feel, ..."
2251,Wanted to be a police officer; after not passi...,0.0,Wanted to be a police officer after not passin...,"[Wanted, to, be, a, police, officer, after, no..."
2316,You don’t and you don’t compare yourself to ot...,0.0,You dont and you dont compare yourself to othe...,"[You, dont, and, you, dont, compare, yourself,..."
2576,I grew up with a high functioning closet alcoh...,0.0,I grew up with a high functioning closet alcoh...,"[I, grew, up, with, a, high, functioning, clos..."
1347,Sounds like anxiety or a panic attack for sure...,0.0,Sounds like anxiety or a panic attack for sure...,"[Sounds, like, anxiety, or, a, panic, attack, ..."


In [10]:
w2v = Word2Vec(df['str_list'].tolist(), min_count=1, vector_size= 300, workers=3, window =3, sg = 1)

In [11]:
kv = w2v.wv

In [12]:
EMBEDDING_DIM = len(kv['heart'])      # we know... it's 300

# initialize embedding matrix and word-to-id map:
embedding_matrix = kv.vectors
vocab_dict = kv.key_to_index

In [13]:
embedding_matrix.shape

(14358, 300)

In [14]:
list(vocab_dict.items())[:20]

[('to', 0),
 ('I', 1),
 ('and', 2),
 ('the', 3),
 ('a', 4),
 ('of', 5),
 ('my', 6),
 ('in', 7),
 ('you', 8),
 ('it', 9),
 ('that', 10),
 ('was', 11),
 ('', 12),
 ('for', 13),
 ('is', 14),
 ('with', 15),
 ('have', 16),
 ('but', 17),
 ('me', 18),
 ('on', 19)]

In [15]:
def sents_to_ids(sentences):
    """
    converting a list of strings to a list of lists of word ids
    """
    text_ids = [vocab_dict[word] for word in sentences]
    return np.asarray(text_ids).astype(np.int64)

In [16]:
df["id_list"] = df["str_list"].apply(sents_to_ids)
# df["outcome_2d"] = df["outcome"].apply(lambda x: np.asarray([x]).astype(np.int64))
X = np.array(df["id_list"])
# y = np.array(df["outcome_2d"])
y = np.array(df['outcome'].tolist()).astype('float32')

In [17]:
df[["str_list", "id_list"]].head()

,str_list,id_list
1997,"[Im, so, so, sorry, for, your, loss, I, feel, ...","[36, 27, 27, 477, 13, 25, 458, 1, 94, 25, 100,..."
2251,"[Wanted, to, be, a, police, officer, after, no...","[4622, 0, 21, 4, 2329, 4711, 110, 26, 203, 3, ..."
2316,"[You, dont, and, you, dont, compare, yourself,...","[153, 47, 2, 8, 47, 2733, 177, 0, 107, 120, 12..."
2576,"[I, grew, up, with, a, high, functioning, clos...","[1, 760, 51, 15, 4, 277, 4026, 2766, 1826, 185..."
1347,"[Sounds, like, anxiety, or, a, panic, attack, ...","[2745, 34, 337, 24, 4, 447, 84, 13, 189, 1545,..."


In [18]:
X = pad_sequences(X).astype('float32') # zero pre-padding

In [19]:
X.shape

(1394, 2286)

In [20]:
MAX_SEQUENCE_LENGTH = X.shape[1]  # Keras' embedding layer expects a specific input length. Padding is often needed here.

embedding_layer = Embedding(embedding_matrix.shape[0],
                            embedding_matrix.shape[1],
                            embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify=y)

# Basic Softmax Classifier

In [22]:
try:
    del reg_model
except:
    pass

reg_model = tf.keras.Sequential()

reg_model.add(embedding_layer)                                        # embedding layer

reg_model.add(tf.keras.layers.GlobalMaxPooling1D())

reg_model.add(Dense(200, activation='relu'))                          # hidden layer

reg_model.add(Dense(1, activation='sigmoid'))                         # classification layer

In [23]:
reg_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2286, 300)         4307400   
_________________________________________________________________
global_max_pooling1d (Global (None, 300)               0         
_________________________________________________________________
dense (Dense)                (None, 200)               60200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 201       
Total params: 4,367,801
Trainable params: 60,401
Non-trainable params: 4,307,400
_________________________________________________________________


In [24]:
reg_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Accuracy(), 
                                                                        tf.keras.metrics.Recall(),
                                                                       tf.keras.metrics.Precision(),
                                                                       tf.keras.metrics.AUC()])
reg_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, verbose=1)

Epoch 1/20
31/31 [==============================] - 5s 104ms/step - loss: 0.7022 - accuracy: 0.0000e+00 - recall: 0.4504 - precision: 0.4791 - auc: 0.4861 - val_loss: 0.6924 - val_accuracy: 0.0000e+00 - val_recall: 0.9619 - val_precision: 0.4988 - val_auc: 0.5616
Epoch 2/20
31/31 [==============================] - 2s 52ms/step - loss: 0.6899 - accuracy: 0.0000e+00 - recall: 0.8300 - precision: 0.5319 - auc: 0.5345 - val_loss: 0.7113 - val_accuracy: 0.0000e+00 - val_recall: 1.0000 - val_precision: 0.5012 - val_auc: 0.6009
Epoch 3/20
31/31 [==============================] - 2s 58ms/step - loss: 0.7082 - accuracy: 0.0000e+00 - recall: 0.6229 - precision: 0.4794 - auc: 0.5318 - val_loss: 0.6858 - val_accuracy: 0.0000e+00 - val_recall: 0.3905 - val_precision: 0.5503 - val_auc: 0.5796
Epoch 4/20
31/31 [==============================] - 2s 61ms/step - loss: 0.6817 - accuracy: 0.0000e+00 - recall: 0.5528 - precision: 0.5956 - auc: 0.6064 - val_loss: 0.6825 - val_accuracy: 0.0000e+00 - val_reca

## Metrics for Basic Softmax Classifier

In [25]:
y_pred_reg_model = reg_model.predict(X_test).round()
print(classification_report(y_test, y_pred_reg_model))

              precision    recall  f1-score   support

         0.0       0.54      0.73      0.62       209
         1.0       0.58      0.37      0.45       210

    accuracy                           0.55       419
   macro avg       0.56      0.55      0.53       419
weighted avg       0.56      0.55      0.53       419



## CNN Model

In [27]:
try:
    del tf_model
except:
    pass

tf_model = tf.keras.Sequential()

tf_model.add(embedding_layer)                                        # embedding layer


tf_model.add(tf.keras.layers.Conv1D(
    filters=5, 
    kernel_size=3, 
    strides=1, 
    padding='same', 
    activation='relu', 
    use_bias=True,
    kernel_initializer='glorot_uniform', 
    bias_initializer='zeros')
            )   

tf_model.add(tf.keras.layers.GlobalMaxPooling1D())

tf_model.add(Dense(100, activation='relu'))                          # hidden layer
tf_model.add(Dense(1, activation='sigmoid'))                         # classification layer

In [28]:
tf_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2286, 300)         4307400   
_________________________________________________________________
conv1d (Conv1D)              (None, 2286, 5)           4505      
_________________________________________________________________
global_max_pooling1d_1 (Glob (None, 5)                 0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               600       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 101       
Total params: 4,312,606
Trainable params: 5,206
Non-trainable params: 4,307,400
_________________________________________________________________


In [29]:
tf_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[tf.keras.metrics.Accuracy(), 
                                                                        tf.keras.metrics.Recall(),
                                                                       tf.keras.metrics.Precision(),
                                                                       tf.keras.metrics.AUC()])


In [30]:
tf_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, verbose=1)


Epoch 1/20
31/31 [==============================] - 6s 167ms/step - loss: 0.6945 - accuracy: 0.0000e+00 - recall_1: 0.7301 - precision_1: 0.4994 - auc_1: 0.4636 - val_loss: 0.6934 - val_accuracy: 0.0000e+00 - val_recall_1: 0.3000 - val_precision_1: 0.5294 - val_auc_1: 0.4848
Epoch 2/20
31/31 [==============================] - 4s 140ms/step - loss: 0.6934 - accuracy: 0.0000e+00 - recall_1: 0.5648 - precision_1: 0.4992 - auc_1: 0.5069 - val_loss: 0.6930 - val_accuracy: 0.0000e+00 - val_recall_1: 0.0048 - val_precision_1: 0.5000 - val_auc_1: 0.5092
Epoch 3/20
31/31 [==============================] - 4s 130ms/step - loss: 0.6923 - accuracy: 0.0000e+00 - recall_1: 0.1096 - precision_1: 0.5416 - auc_1: 0.5633 - val_loss: 0.6923 - val_accuracy: 0.0000e+00 - val_recall_1: 0.7524 - val_precision_1: 0.5215 - val_auc_1: 0.5479
Epoch 4/20
31/31 [==============================] - 5s 146ms/step - loss: 0.6921 - accuracy: 0.0000e+00 - recall_1: 0.6492 - precision_1: 0.5275 - auc_1: 0.5426 - val_loss:

In [31]:
y_pred_tf_model = tf_model.predict(X_test).round()
print(y_pred_tf_model.shape)
print(classification_report(y_test, y_pred_tf_model))

(419, 1)
              precision    recall  f1-score   support

         0.0       0.55      0.56      0.56       209
         1.0       0.55      0.53      0.54       210

    accuracy                           0.55       419
   macro avg       0.55      0.55      0.55       419
weighted avg       0.55      0.55      0.55       419

